### Calculate connections between an ego and its channels through neighbors 

In [ ]:
import pandas as pd
import networkx as nx
from collections import Counter
import matplotlib.pyplot as plt
import igraph as ig
import numpy as np

from networkx.algorithms.community import louvain_communities

In [ ]:
def get_graph(useruser, userchannel):
    uucg = nx.DiGraph()
    uucg.add_nodes_from(useruser.userid.unique())
    uucg.add_nodes_from(useruser.neighbourid.unique())
    uucg.add_nodes_from(userchannel.channelid.unique())
    
    for i, row in useruser.iterrows():
        uucg.add_edge(row.userid, row.neighbourid, weight=row.score)
    
    for i, row in userchannel.iterrows():
        uucg.add_edge(row.userid, row.channelid, weight=row.score)
        
    print(uucg)
    return uucg

In [ ]:
def get_ego_connections(useruser, userchannel, uucg): 
    users = np.unique(np.concatenate((useruser.userid.unique(), useruser.neighbourid.unique()), axis=0)).tolist()
    channels = userchannel.channelid.unique()

    useruserchannel = []
    
    for i, node in enumerate(uucg.nodes):

        if node not in users:
            continue

        print(f"Working on node {i}", end="\r")

        ego = node

        # Direct connections between the ego and channels with new values
        new_connections = {}
        # List of all ego neighbors 
        all_neighbours = {}
        # List of all channels the ego is connected to
        all_channels = {}

        for neighbor in uucg.neighbors(ego):
            if neighbor in users:
                all_neighbours[neighbor] = uucg[ego][neighbor]["weight"]

            if neighbor in channels:
                all_channels[neighbor] = uucg[ego][neighbor]["weight"]

        # Get connections between neighbors and channels and calcualte the ego neighbor channel connection weight
        for neighbor in all_neighbours.keys():
            for nn in uucg.neighbors(neighbor):
                if nn in all_channels.keys():
                    if nn in new_connections.keys():
                        new_connections[nn]["score"] += 0.5 * all_neighbours[neighbor] + 0.5 * uucg[neighbor][nn]["weight"]
                        # We will later use this value to normalise
                        new_connections[nn]["connection_count"] += 1
                    else:
                        new_connections[nn] = {
                            "score": 0.5 * all_neighbours[neighbor] + 0.5 * uucg[neighbor][nn]["weight"],
                            "connection_count": 1
                        }

        for channel in new_connections.keys():  
            useruserchannel.append({
                "userid": ego,
                "channelid": channel,
                "weight": new_connections[channel]["score"] / new_connections[channel]["connection_count"]
            }) 

    return useruserchannel

In [ ]:
user_user_conf = [
    {"j": 0, "k": 0, "l": 1}
]


for uu_conf in user_user_conf:
    print("Reading user-user")
    useruser = pd.read_pickle(f"./data/final/user-user-{uu_conf['j']}-{uu_conf['k']}-{uu_conf['l']}.pkl")

    for j in [0,0.5,1]:
        for k in [0,0.5,1]:
            print(f"Reading user-channel conf: {uu_conf=} {j=} {k=}")
            userchannel = pd.read_pickle(f"./data/final/user-channel-{j}-{k}.pkl")
            print("Getting graph")
            uucg = get_graph(useruser, userchannel)
            print("Getting new connections")
            ego_channel = get_ego_connections(useruser, userchannel,uucg)
            print("Generating dataframe")
            ego_channel_df = pd.DataFrame(ego_channel)
            print(ego_channel_df.head(5))
            ego_channel_df.to_pickle(f"./data/final/ego-channel-{j}-{k}-with-user-conf-{uu_conf['j']}-{uu_conf['k']}-{uu_conf['l']}.pkl")